In [49]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
from sklearn.metrics import accuracy_score,classification_report,hamming_loss,f1_score,precision_score,recall_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from collections import Counter
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import wandb

In [50]:
# wandb.login(key="a3ab9e880247c3575c105c3a5abb88ebc2eba87a")

### Splitting data into train, test and validation sets

In [51]:
data=pd.read_csv('WineQT.csv')
X=data.loc[:,~data.columns.isin(['quality','Id'])].values
y=data['quality'].values
temp=np.zeros((y.shape[0],6), dtype=int)
temp[np.arange(y.shape[0]), y - 3] = 1
y_onehot=temp
x_train, x_temp, y_train, y_temp = train_test_split(X, y_onehot, test_size=0.3, random_state=42)
x_val, x_test, y_val, y_test = train_test_split(x_temp, y_temp, test_size=0.5, random_state=42)

### Standardizing and Normalising Data

In [52]:
scalar=StandardScaler()
scalar.fit(x_train)
x_train=scalar.transform(x_train)
x_val=scalar.transform(x_val)
x_test=scalar.transform(x_test)
minmax_scalar=MinMaxScaler()
minmax_scalar.fit(x_train)
x_train=minmax_scalar.transform(x_train)
x_val=minmax_scalar.transform(x_val)
x_test=minmax_scalar.transform(x_test)

In [53]:
class MLPClassifier:
    def __init__(self,learning_rate,activation_function,optimizer,num_hidden_layers,count_neurons):
        self.lrate=learning_rate
        self.activation_func=activation_function
        self.optimizer=optimizer
        self.hidden_layers=num_hidden_layers
        self.num_neurons=count_neurons
        self.input_layer_size=None
        self.ouput_layer_size=None
        self.weights=None
        self.biases=None
        self.batch_size=100
    def set_activation_function(self,af):
        self.activation_func=af
    def set_optimizer(self,opt):
        self.optimizer=opt
    def set_learning_rate(self,lr):
        self.lrate=lr
    def set_hidden_layers(self,hl):
        self.hidden_layers=hl
    def set_num_neurons(self,m):
        self.num_neurons=m
    def ReLu(self,z):
        return np.maximum(0,z)
    def Tanh(self,z):
        return (2/(1+np.exp(-2*z)))-1
    def Sigmoid(self,z):
        return 1/(1+np.exp(-z))
    def Softmax(self,z):
        expz=np.exp(z)
        return expz/np.sum(expz,axis=1,keepdims=True)
    def init_weights_biases(self):
        weights=[]
        biases=[]
        for i in range(1,len(self.layer_sizes)):
            weight_matrix = np.random.randn(self.layer_sizes[i-1],self.layer_sizes[i]) * math.sqrt(2.0/self.layer_sizes[i-1])
            bias_vector = np.zeros((1,self.layer_sizes[i]))
            weights.append(weight_matrix)
            biases.append(bias_vector)
        return weights, biases
    def forward_propagation(self,X):
        activations = [X]
        z_values = []
        for i in range(len(self.layer_sizes) - 1):
            z = activations[-1]@self.weights[i] + self.biases[i]
            a = self.Sigmoid(z) if i < len(self.layer_sizes) - 2 else self.Softmax(z)
            if self.activation_func=='tanh':
                a = self.Tanh(z) if i < len(self.layer_sizes) - 2 else self.Softmax(z)
            elif self.activation_func=='ReLu':
                a = self.ReLu(z) if i < len(self.layer_sizes) - 2 else self.Softmax(z)
            z_values.append(z)
            activations.append(a) 
        return activations, z_values
    def backward_propagation(self,y,activations):
        grads = []
        delta = activations[-1] - y
        for i in range(len(self.layer_sizes) - 2, -1, -1):
            dw = activations[i].T@delta
            db = np.sum(delta, axis=0, keepdims=True)
            grads.append((dw, db))
            if i > 0:
                if self.activation_func=='sigmoid':
                    delta = (delta@self.weights[i].T) * activations[i] * (1 - activations[i])
                elif self.activation_func=='tanh':
                    delta=(delta@self.weights[i].T)*(1-np.square(activations[i]))
                else:
                    delta=(delta@self.weights[i].T)*((activations[i]>0).astype(int))
        grads.reverse()
        return grads
    def compute_loss(self,y,y_pred):
        return -np.sum(y * np.log(y_pred))
    def update_parameters(self,grads):
        for i in range(len(self.weights)):
            self.weights[i] -= self.lrate * grads[i][0]
            self.biases[i] -= self.lrate * grads[i][1]
    def train(self,x,y,x_val,y_val,num_epochs):
        # nm=f"LR:{self.lrate},HL:{self.hidden_layers},E:{num_epochs},HLN:{self.num_neurons}"
        # wandb.init(
        #     project="MLPClassifier-EHLHLNLR",
        #     config={
        #         "Learning Rate":self.lrate,
        #         "Epochs":num_epochs,
        #         "Optimizer":self.optimizer,
        #         "Layers":self.hidden_layers,
        #         "LayerSize":self.num_neurons
        #     },
        #     name=nm
        # )
        self.input_layer_size=x.shape[1]
        self.ouput_layer_size=y.shape[1]
        self.layer_sizes=[self.input_layer_size]+self.hidden_layers*[self.num_neurons]+[self.ouput_layer_size]
        self.weights,self.biases=self.init_weights_biases()
        m=x.shape[0]
        if self.optimizer=='mbgd':
            for _ in range(num_epochs):
                permutation = np.random.permutation(m)
                x_shuffled = x[permutation,:]
                y_shuffled = y[permutation,:]
                for i in range(0, m, self.batch_size):
                    x_batch = x_shuffled[i:i + self.batch_size,:]
                    y_batch = y_shuffled[i:i + self.batch_size,:]
                    if len(x_batch) < self.batch_size:
                        remaining_samples =m % self.batch_size
                        x_batch = x_shuffled[i:i+remaining_samples]
                        y_batch = y_shuffled[i:i+remaining_samples]
                    activations,_ = self.forward_propagation(x_batch)
                    loss = self.compute_loss(y_batch, activations[-1])
                    self.final_loss=loss
                    grads = self.backward_propagation(y_batch, activations)
                    self.update_parameters(grads)
                # activations, _ = self.forward_propagation(x_val)
                # pred_val=activations[-1]
                # loss_val=self.compute_loss(y_val,pred_val)
                # max_indices = np.argmax(pred_val, axis=1).reshape(-1, 1)
                # new_matrix = np.zeros((max_indices.shape[0], 6), dtype=int)
                # rows, cols = np.arange(max_indices.shape[0]), max_indices.flatten()
                # new_matrix[rows, cols] = 1
                # accuracy_val=accuracy_score(y_val,new_matrix)
                # activations, _ = self.forward_propagation(x)
                # pred_train=activations[-1]
                # loss_train=self.compute_loss(y,pred_train)
                # max_indices = np.argmax(pred_train, axis=1).reshape(-1, 1)
                # new_matrix = np.zeros((max_indices.shape[0], 6), dtype=int)
                # rows, cols = np.arange(max_indices.shape[0]), max_indices.flatten()
                # new_matrix[rows, cols] = 1
                # accuracy_train=accuracy_score(y,new_matrix)
                # wandb.log({'val_loss':loss_val,'train_loss':loss_train,'train_accuracy':accuracy_train,'val_accuracy':accuracy_val})
        elif self.optimizer=='gd':
            for _ in range(num_epochs):
                activations,_ = self.forward_propagation(x)
                loss = self.compute_loss(y, activations[-1])
                self.final_loss=loss
                grads = self.backward_propagation(y, activations)
                self.update_parameters(grads)
                # activations, _ = self.forward_propagation(x_val)
                # pred_val=activations[-1]
                # loss_val=self.compute_loss(y_val,pred_val)
                # max_indices = np.argmax(pred_val, axis=1).reshape(-1, 1)
                # new_matrix = np.zeros((max_indices.shape[0], 6), dtype=int)
                # rows, cols = np.arange(max_indices.shape[0]), max_indices.flatten()
                # new_matrix[rows, cols] = 1
                # accuracy_val=accuracy_score(y_val,new_matrix)
                # activations, _ = self.forward_propagation(x)
                # pred_train=activations[-1]
                # loss_train=self.compute_loss(y,pred_train)
                # max_indices = np.argmax(pred_train, axis=1).reshape(-1, 1)
                # new_matrix = np.zeros((max_indices.shape[0], 6), dtype=int)
                # rows, cols = np.arange(max_indices.shape[0]), max_indices.flatten()
                # new_matrix[rows, cols] = 1
                # accuracy_train=accuracy_score(y,new_matrix)
                # wandb.log({'val_loss':loss_val,'train_loss':loss_train,'train_accuracy':accuracy_train,'val_accuracy':accuracy_val})
        else:
            for _ in range(num_epochs):
                permutation = np.random.permutation(m)
                x_shuffled = x[permutation,:]
                y_shuffled = y[permutation,:]
                for i in range(m):
                    x_sample = x_shuffled[i:i+1,:]
                    y_sample = y_shuffled[i:i+1,:]
                    activations,_ = self.forward_propagation(x_sample)
                    loss = self.compute_loss(y_sample, activations[-1])
                    self.final_loss=loss
                    grads = self.backward_propagation(y_sample, activations)
                    self.update_parameters(grads)
                # activations, _ = self.forward_propagation(x_val)
                # pred_val=activations[-1]
                # loss_val=self.compute_loss(y_val,pred_val)
                # max_indices = np.argmax(pred_val, axis=1).reshape(-1, 1)
                # new_matrix = np.zeros((max_indices.shape[0], 6), dtype=int)
                # rows, cols = np.arange(max_indices.shape[0]), max_indices.flatten()
                # new_matrix[rows, cols] = 1
                # accuracy_val=accuracy_score(y_val,new_matrix)
                # activations, _ = self.forward_propagation(x)
                # pred_train=activations[-1]
                # loss_train=self.compute_loss(y,pred_train)
                # max_indices = np.argmax(pred_train, axis=1).reshape(-1, 1)
                # new_matrix = np.zeros((max_indices.shape[0], 6), dtype=int)
                # rows, cols = np.arange(max_indices.shape[0]), max_indices.flatten()
                # new_matrix[rows, cols] = 1
                # accuracy_train=accuracy_score(y,new_matrix)
                # wandb.log({'val_loss':loss_val,'train_loss':loss_train,'train_accuracy':accuracy_train,'val_accuracy':accuracy_val})
    def predict(self,x):
        activations, _ = self.forward_propagation(x)
        return activations[-1]

### Model Evaluation

In [54]:
cl=MLPClassifier(0.0001,'sigmoid','gd',2,7)
cl.train(x_train,y_train,x_val,y_val,10000)
l=cl.predict(x_test)
max_indices = np.argmax(l, axis=1).reshape(-1, 1)
new_matrix = np.zeros((max_indices.shape[0], 6), dtype=int)
rows, cols = np.arange(max_indices.shape[0]), max_indices.flatten()
new_matrix[rows, cols] = 1
print(f'Accuracy Score on test set : {accuracy_score(y_test,new_matrix)}')
y_trueaccrep=np.argmax(y_test, axis=1).reshape(-1, 1)+3
print(classification_report(y_trueaccrep,max_indices+3,zero_division=True))

Accuracy Score on test set : 0.5988372093023255
              precision    recall  f1-score   support

           4       1.00      0.00      0.00         6
           5       0.69      0.73      0.71        81
           6       0.53      0.63      0.57        65
           7       0.38      0.18      0.24        17
           8       1.00      0.00      0.00         3

    accuracy                           0.60       172
   macro avg       0.72      0.31      0.30       172
weighted avg       0.61      0.60      0.57       172



In [55]:
# lrates=[0.001,0.0001]
# epochs=[5000,10000]
# hln=[4,7]
# hl=[2,4]
# for lra in lrates:
#     for ep in epochs:
#         for a in hln:
#             for b in hl:
#                 cl=MLPClassifier(lra,'sigmoid','gd',b,a)
#                 cl.train(x_train,y_train,x_val,y_val,ep)
#                 l=cl.predict(x_test)
#                 max_indices = np.argmax(l, axis=1).reshape(-1, 1)
#                 new_matrix = np.zeros((max_indices.shape[0], 6), dtype=int)
#                 rows, cols = np.arange(max_indices.shape[0]), max_indices.flatten()
#                 new_matrix[rows, cols] = 1
#                 wandb.log({'test_accuracy':100*accuracy_score(y_test,new_matrix),'test_f1score':100*f1_score(y_test,new_matrix,average="macro"),'test_recall':100*recall_score(y_test,new_matrix,average="macro"),'test_precision':100*precision_score(y_test,new_matrix,average="macro")})
# wandb.finish()

In [56]:
# afs=['sigmoid','ReLu','tanh']
# opt=['gd','sgd','mbgd']
# for o in opt:
#     for af in afs:
#         cl=MLPClassifier(0.001,af,o,2,10)
#         cl.train(x_train,y_train,x_val,y_val,5000)
#         l=cl.predict(x_test)
#         max_indices = np.argmax(l, axis=1).reshape(-1, 1)
#         new_matrix = np.zeros((max_indices.shape[0], 6), dtype=int)
#         rows, cols = np.arange(max_indices.shape[0]), max_indices.flatten()
#         new_matrix[rows, cols] = 1
#         wandb.log({'test_accuracy':100*accuracy_score(y_test,new_matrix),'test_f1score':100*f1_score(y_test,new_matrix,average="macro"),'test_recall':100*recall_score(y_test,new_matrix,average="macro"),'test_precision':100*precision_score(y_test,new_matrix,average="macro")})
# wandb.finish()

##### The MLP classifier gives a slightly better accuracy than the multinomial logistic regression classifier.

### Multi-Label Classification

In [57]:
data=pd.read_csv('advertisement.csv')
data=data.drop('city', axis=1)
x=data.iloc[:, :-1]
y=data.iloc[:, -1].values

In [58]:
def wtoi(y):
    vis=""
    for s in y:
        vis+=s
        vis+=" "
    words = vis.split()
    word_counts = Counter(words) 
    word_list = list(word_counts.keys())
    uniq_labels=sorted(word_list)
    return {word: index for index, word in enumerate(uniq_labels)}

def encode_labels(y):
        word_to_index=wtoi(y)
        vis=""
        for s in y:
            vis+=s
            vis+=" "
        words = vis.split()
        word_counts = Counter(words) 
        word_list = list(word_counts.keys())
        uniq_labels=sorted(word_list)
        uniq_count=len(uniq_labels)
        word_to_index = {word: index for index, word in enumerate(uniq_labels)}
        vecs=[]
        for spl in y:
            new_label=[0]*uniq_count
            present=spl.split()
            for w in present:
                new_label[word_to_index[w]]=1
            vecs.append(new_label)
        return np.array(vecs)

y_onehot=encode_labels(y)

def onehot(X):
    cols=['gender','education','married','occupation','most bought item']
    x_new=pd.get_dummies(X,columns=cols)
    return x_new.values

x_onehot=onehot(x)
x_onehot[:,4:]=x_onehot[:,4:].astype(float)
mms=MinMaxScaler()
x_train,x_test,y_train,y_test=train_test_split(x_onehot,y_onehot,test_size=0.3,random_state=42)
x_test,x_val,y_test,y_val=train_test_split(x_test,y_test,test_size=0.5,random_state=42)
x_train=mms.fit_transform(x_train)
x_test=mms.transform(x_test)
x_val=mms.transform(x_val)

In [59]:
class MLPMultiLabelClassifier:
    def __init__(self,learning_rate,activation_function,optimizer,num_hidden_layers,count_neurons):
        self.lrate=learning_rate
        self.activation_func=activation_function
        self.optimizer=optimizer
        self.hidden_layers=num_hidden_layers
        self.num_neurons=count_neurons
        self.input_layer_size=None
        self.ouput_layer_size=None
        self.weights=None
        self.biases=None
        self.batch_size=100
    def set_activation_function(self,af):
        self.activation_func=af
    def set_optimizer(self,opt):
        self.optimizer=opt
    def set_learning_rate(self,lr):
        self.lrate=lr
    def set_hidden_layers(self,hl):
        self.hidden_layers=hl
    def set_num_neurons(self,m):
        self.num_neurons=m
    def ReLu(self,z):
        return np.maximum(0,z)
    def Tanh(self,z):
        return (2/(1+np.exp(-2*z)))-1
    def Sigmoid(self,z):
        return 1/(1+np.exp(-z))
    def init_weights_biases(self):
        weights=[]
        biases=[]
        for i in range(1,len(self.layer_sizes)):
            weight_matrix = np.random.randn(self.layer_sizes[i-1],self.layer_sizes[i]) * math.sqrt(2.0/self.layer_sizes[i-1])
            bias_vector = np.zeros((1,self.layer_sizes[i]))
            weights.append(weight_matrix)
            biases.append(bias_vector)
        return weights, biases
    def forward_propagation(self,X):
        activations = [X]
        z_values = []
        for i in range(len(self.layer_sizes) - 1):
            z = activations[-1]@self.weights[i] + self.biases[i]
            a = self.Sigmoid(z)
            if self.activation_func=='tanh':
                a = self.Tanh(z) if i < len(self.layer_sizes) - 2 else self.Sigmoid(z)
            elif self.activation_func=='ReLu':
                a = self.ReLu(z) if i < len(self.layer_sizes) - 2 else self.Sigmoid(z)
            z_values.append(z)
            activations.append(a) 
        return activations, z_values
    def backward_propagation(self,y,activations):
        grads = []
        delta = activations[-1] - y
        for i in range(len(self.layer_sizes) - 2, -1, -1):
            dw = activations[i].T@delta
            db = np.sum(delta, axis=0, keepdims=True)
            grads.append((dw, db))
            if i > 0:
                if self.activation_func=='sigmoid':
                    delta = (delta@self.weights[i].T) * activations[i] * (1 - activations[i])
                elif self.activation_func=='tanh':
                    delta=(delta@self.weights[i].T)*(1-np.square(activations[i]))
                else:
                    delta=(delta@self.weights[i].T)*((activations[i]>0).astype(int))
        grads.reverse()
        return grads
    def compute_loss(self,y,y_pred):
        return -np.sum(y * np.log(y_pred))
    def update_parameters(self,grads):
        for i in range(len(self.weights)):
            self.weights[i] -= self.lrate * grads[i][0]
            self.biases[i] -= self.lrate * grads[i][1]
    def train(self,x,y,x_val,y_val,num_epochs):
        # nm=f"LR:{self.lrate},HL:{self.hidden_layers},HLN:{self.num_neurons},E:{num_epochs}"
        # wandb.init(
        #     project="MultiLabel-MLP-EHLHLNLR",
        #     config={
        #         "Learning Rate":self.lrate,
        #         "Epochs":num_epochs,
        #         "Optimizer":self.optimizer,
        #         "Layers":self.hidden_layers,
        #         "LayerSize":self.num_neurons
        #     },
        #     name=nm
        # )
        self.input_layer_size=x.shape[1]
        self.ouput_layer_size=y.shape[1]
        self.layer_sizes=[self.input_layer_size]+self.hidden_layers*[self.num_neurons]+[self.ouput_layer_size]
        self.weights,self.biases=self.init_weights_biases()
        m=x.shape[0]
        if self.optimizer=='mbgd':
            for _ in range(num_epochs):
                permutation = np.random.permutation(m)
                x_shuffled = x[permutation,:]
                y_shuffled = y[permutation,:]
                for i in range(0, m, self.batch_size):
                    x_batch = x_shuffled[i:i + self.batch_size,:]
                    y_batch = y_shuffled[i:i + self.batch_size,:]
                    if len(x_batch) < self.batch_size:
                        remaining_samples =m % self.batch_size
                        x_batch = x_shuffled[i:i+remaining_samples]
                        y_batch = y_shuffled[i:i+remaining_samples]
                    activations,_ = self.forward_propagation(x_batch)
                    loss = self.compute_loss(y_batch, activations[-1])
                    self.final_loss=loss
                    grads = self.backward_propagation(y_batch, activations)
                    self.update_parameters(grads)
                # activations, _ = self.forward_propagation(x_val)
                # pred_val=activations[-1]
                # loss_val=self.compute_loss(y_val,pred_val)
                # pred_val = (pred_val > 0.5).astype(int)
                # accuracy_val=accuracy_score(y_val,pred_val)
                # activations, _ = self.forward_propagation(x)
                # pred_train=activations[-1]
                # loss_train=self.compute_loss(y,pred_train)
                # pred_train=(pred_train > 0.5).astype(int)
                # accuracy_train=accuracy_score(y,pred_train)
                # wandb.log({'val_loss':loss_val,'train_loss':loss_train,'train_accuracy':accuracy_train,'val_accuracy':accuracy_val})
        elif self.optimizer=='gd':
            for _ in range(num_epochs):
                activations,_ = self.forward_propagation(x)
                loss = self.compute_loss(y, activations[-1])
                self.final_loss=loss
                grads = self.backward_propagation(y, activations)
                self.update_parameters(grads)
                # logging
                # activations, _ = self.forward_propagation(x_val)
                # pred_val=activations[-1]
                # loss_val=self.compute_loss(y_val,pred_val)
                # pred_val = (pred_val > 0.5).astype(int)
                # accuracy_val=accuracy_score(y_val,pred_val)
                # activations, _ = self.forward_propagation(x)
                # pred_train=activations[-1]
                # loss_train=self.compute_loss(y,pred_train)
                # pred_train=(pred_train > 0.5).astype(int)
                # accuracy_train=accuracy_score(y,pred_train)
                # wandb.log({'val_loss':loss_val,'train_loss':loss_train,'train_accuracy':accuracy_train,'val_accuracy':accuracy_val})
        else:
            for _ in range(num_epochs):
                permutation = np.random.permutation(m)
                x_shuffled = x[permutation,:]
                y_shuffled = y[permutation,:]
                for i in range(m):
                    x_sample = x_shuffled[i:i+1,:]
                    y_sample = y_shuffled[i:i+1,:]
                    activations,_ = self.forward_propagation(x_sample)
                    loss = self.compute_loss(y_sample, activations[-1])
                    self.final_loss=loss
                    grads = self.backward_propagation(y_sample, activations)
                    self.update_parameters(grads)
                #logging
                # activations, _ = self.forward_propagation(x_val)
                # pred_val=activations[-1]
                # loss_val=self.compute_loss(y_val,pred_val)
                # pred_val = (pred_val > 0.5).astype(int)
                # accuracy_val=accuracy_score(y_val,pred_val)
                # activations, _ = self.forward_propagation(x)
                # pred_train=activations[-1]
                # loss_train=self.compute_loss(y,pred_train)
                # pred_train=(pred_train > 0.5).astype(int)
                # accuracy_train=accuracy_score(y,pred_train)
                # wandb.log({'val_loss':loss_val,'train_loss':loss_train,'train_accuracy':accuracy_train,'val_accuracy':accuracy_val})
    def predict(self,x):
        activations, _ = self.forward_propagation(x)
        return activations[-1]

In [60]:
cl=MLPMultiLabelClassifier(0.001,'tanh','mbgd',3,8)
cl.train(x_train,y_train,x_val,y_val,5000)
l=cl.predict(x_test)
pred = (l > 0.5).astype(int)
print(f'Accuracy Score for test set : {100*accuracy_score(y_test,pred)}%')
print(f'F1-Score for test set : {f1_score(y_test,pred,average="macro")}')
print(f'Recall Score for test set : {recall_score(y_test,pred,average="macro")}')
print(f'Precision Score for test set : {precision_score(y_test,pred,average="macro")}')

Accuracy Score for test set : 5.333333333333334%
F1-Score for test set : 0.5259849718034559
Recall Score for test set : 0.5126408882439127
Precision Score for test set : 0.5539904831365337


In [61]:
# lrates=[0.0001,0.00001]
# epochs=[5000,10000]
# hln=[8,10]
# hl=[2,4]
# for lra in lrates:
#     for ep in epochs:
#         for a in hln:
#             for b in hl:
#                 cl=MLPMultiLabelClassifier(lra,'sigmoid','gd',b,a)
#                 cl.train(x_train,y_train,x_val,y_val,ep)
#                 l=cl.predict(x_test)
#                 pred = (l > 0.5).astype(int)
#                 wandb.log({'test_accuracy':100*accuracy_score(y_test,pred),'test_f1score':100*f1_score(y_test,pred,average="macro"),'test_recall':100*recall_score(y_test,pred,average="macro"),'test_precision':100*precision_score(y_test,pred,average="macro")})
# wandb.finish()

In [62]:
# afs=['sigmoid','ReLu','tanh']
# opt=['gd','sgd','mbgd']
# for o in opt:
#     for af in afs:
#         cl=MLPMultiLabelClassifier(0.001,af,o,2,10)
#         cl.train(x_train,y_train,x_val,y_val,5000)
#         l=cl.predict(x_test)
#         pred = (l > 0.5).astype(int)
#         wandb.log({'test_accuracy':100*accuracy_score(y_test,pred),'test_f1score':100*f1_score(y_test,pred,average="macro"),'test_recall':100*recall_score(y_test,pred,average="macro"),'test_precision':100*precision_score(y_test,pred,average="macro")})
# wandb.finish()